### Creating a logistic regression to classify Bayern vs. Dortmund

In [ ]:
import numpy as np
import pandas as pd
from statsbombpy import sb
pd.set_option('display.max_columns', None)
from mplsoccer import Pitch, Sbopen
from mplsoccer import VerticalPitch,Pitch
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import math 


In [ ]:
def digraph(data):
        G = nx.DiGraph()
        pass_counts = {}
        players = data["player_name"].unique()
        G.add_nodes_from(players)
        for _, row in data.iterrows():
                key = (row["player_name"], row["pass_recipient_name"])
                if key in pass_counts:
                        pass_counts[key] += 1
                else:
                        pass_counts[key] = 1

        for key, weight in pass_counts.items():
                G.add_edge(key[0], key[1], weight=weight)
        
        return G

In [ ]:
parser = Sbopen()
df_competitions = parser.competition()
df_competitions

In [ ]:
teams = parser.match(competition_id=7, season_id=27)


##  Pseudo code
make data frame
Y- team with X1 = between centrality ...
- loop through df_match for each match
- if statement for same game just remove
    
- if away_team_name == Bayern Munich or Borussia Dortmund
    append team name
- make digraph

- compute centrality measures


In [ ]:
df.columns

In [164]:
def pass_data_monaco(match_id):
    df, related, freeze, tactics = parser.event(match_id)
    sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "AS Monaco"].iloc[0]["index"]

    mask_monaco = (df.type_name == 'Pass') & (df.team_name == "AS Monaco") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
    df_pass = df.loc[mask_monaco, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
    df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
    df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])
    return df_pass  



In [ ]:
def pass_data_lyon(match_id):
    df, related, freeze, tactics = parser.event(match_id)

    sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "Lyon"].iloc[0]["index"]
    mask_lyon = (df.type_name == 'Pass') & (df.team_name == "Lyon") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
    df_pass = df.loc[mask_lyon, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
    df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
    df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])
    return df_pass

In [165]:


def get_data(dat):
    data = pd.DataFrame()
    for index, row in dat.iterrows():
        if (row['home_team_name'] == 'AS Monaco'):
            match_id = row['match_id']
            passing_data_monaco = pass_data_monaco(match_id)
            
            G = digraph(passing_data_monaco)
            betweenness_centrality = (nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).items(), key=lambda x: x[1])[1]
            katz_centrality = max(nx.katz_centrality(G).items(), key=lambda x: x[1])[1]
            page_rank = max(nx.pagerank(G).items(), key=lambda x: x[1])[1]
            
            match_data = {
                'Match ID': match_id,
                'Team': 'AS Monaco',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)
        if (row['away_team_name'] == 'AS Monaco'):
            match_id = row['match_id']
            passing_data_monaco = pass_data_monaco(match_id)
            G = digraph(passing_data_monaco)
            betweenness_centrality = max(nx.betweenness_centrality(G).items(), key=lambda x: x[1])[1]
            eigenvector_centrality = max(nx.eigenvector_centrality(G).items(), key=lambda x: x[1])[1]
            katz_centrality = max(nx.katz_centrality(G).items(), key=lambda x: x[1])[1]
            page_rank = max(nx.pagerank(G).items(), key=lambda x: x[1])[1]
            match_data = {
                'Match ID': match_id,
                'Team': 'AS Monaco',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)

        if (row['home_team_name'] == 'Lyon'):
            match_id = row['match_id']
            passing_data_lyon = pass_data_lyon(match_id=match_id)
            G = digraph(passing_data_lyon)
            betweenness_centrality = max(nx.betweenness_centrality(G).items(), key=lambda x: x[1])[1]
            eigenvector_centrality = max(nx.eigenvector_centrality(G).items(), key=lambda x: x[1])[1]
            katz_centrality = max(nx.katz_centrality(G).items(), key=lambda x: x[1])[1]
            page_rank = max(nx.pagerank(G).items(), key=lambda x: x[1])[1]
            match_data = {
                'Match ID': match_id,
                'Team': 'Lyon',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)
        if (row['away_team_name'] == 'Lyon'):
            match_id = row['match_id']
            passing_data_lyon = pass_data_lyon(match_id=match_id)
            G = digraph(passing_data_lyon)
            betweenness_centrality = max(nx.betweenness_centrality(G).items(), key=lambda x: x[1])[1]
            eigenvector_centrality = max(nx.eigenvector_centrality(G).items(), key=lambda x: x[1])[1]
            katz_centrality = max(nx.katz_centrality(G).items(), key=lambda x: x[1])[1]
            page_rank = max(nx.pagerank(G).items(), key=lambda x: x[1])[1]
            match_data = {
                'Match ID': match_id,
                'Team': 'Lyon',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)

    return data

d = get_data(dat=teams)
d


C:\Users\Barre\AppData\Local\Temp/ipykernel_2612/4031219927.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(match_data, ignore_index=True)
C:\Users\Barre\AppData\Local\Temp/ipykernel_2612/4031219927.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(match_data, ignore_index=True)
C:\Users\Barre\AppData\Local\Temp/ipykernel_2612/4031219927.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(match_data, ignore_index=True)
C:\Users\Barre\AppData\Local\Temp/ipykernel_2612/4031219927.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(match_data, ignore_inde

,Match ID,Team,Betweenness Centrality,Eigenvector Centrality,Katz Centrality,Page Rank,Average Clustering
0,3900587,Lyon,0.056455,0.349025,0.339700,0.116283,0.817255
1,3900541,AS Monaco,0.129815,0.414477,0.364726,0.194167,0.542119
2,3901256,AS Monaco,0.076111,0.403196,0.378183,0.140889,0.712997
3,3901254,Lyon,0.190185,0.435885,0.380932,0.142315,0.604859
4,3901252,AS Monaco,0.160741,0.372350,0.349181,0.134622,0.600437
5,3901252,Lyon,0.052249,0.350423,0.340628,0.125333,0.766211
6,3901238,AS Monaco,0.129444,0.386562,0.356061,0.150495,0.630341
7,3901235,Lyon,0.067963,0.356812,0.349660,0.130499,0.849281
8,3901231,AS Monaco,0.079365,0.367042,0.351722,0.140814,0.729137
9,3901225,Lyon,0.047169,0.345351,0.340418,0.128391,0.871777


In [189]:
X = d.drop(['Team','Match ID'], axis=1)
y = d['Team']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)



LogisticRegression()

In [190]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [191]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

Accuracy: 37.50%
Confusion Matrix:
 [[0 9]
 [1 6]]
Classification Report:
               precision    recall  f1-score   support

   AS Monaco       0.00      0.00      0.00         9
        Lyon       0.40      0.86      0.55         7

    accuracy                           0.38        16
   macro avg       0.20      0.43      0.27        16
weighted avg       0.18      0.38      0.24        16

